Connect to the workspace

In [1]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#Enter details of your Azure Machine Learning workspace
subscription_id = ""
resource_group = ""
workspace = ""

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

Creating compute cluster

In [4]:
from azure.ai.ml.entities import AmlCompute

cluster = AmlCompute(
    name="cluster-with-1-k80-gpu",
    type="amlcompute",
    size="Standard_NC6",
    location="westeurope",
    min_instances=0,l
    max_instances=2,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster).result()

In [5]:
#import requests library
import requests

#define the url of the weights file
url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8x-seg.pt"

#define the path to save the weights file in your workspace folder
path = "yolov8x-seg.pt"

#download the weights file using requests
response = requests.get(url, stream=True)

#check if the response is successful
if response.status_code == 200:
    #open the file in write mode
    with open(path, "wb") as f:
        #write the response content to the file
        f.write(response.content)
    #print a success message
    print(f"Downloaded {url} and saved as {path}")
else:
    #print an error message
    print(f"Failed to download {url}")


Create a compute resource for training

In [6]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster-train"

try:
    ml_client.compute.get(cpu_compute_target)
    print("Found existing compute target.")
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="Standard_D13_v2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()


Create an AzureML environment

In [7]:

%%bash
mkdir ext-env

echo """
FROM pytorch/pytorch:2.0.0-cuda11.7-cudnn8-runtime

# Downloads to user config dir
ADD https://ultralytics.com/assets/Arial.ttf https://ultralytics.com/assets/Arial.Unicode.ttf /root/.config/Ultralytics/

# Install linux packages
ENV DEBIAN_FRONTEND noninteractive
RUN apt update
RUN TZ=Etc/UTC apt install -y tzdata
RUN apt install --no-install-recommends -y gcc git zip curl htop libgl1-mesa-glx libglib2.0-0 libpython3-dev gnupg g++

# Security updates
# https://security.snyk.io/vuln/SNYK-UBUNTU1804-OPENSSL-3314796
RUN apt upgrade --no-install-recommends -y openssl tar

RUN pip install ultralytics==8.0.133
RUN pip install azureml-mlflow==1.52.0
RUN pip install mlflow==2.4.2
RUN pip install azureml-core==1.52.0
RUN pip install azureml-dataprep==4.12.1
RUN pip install pip install fuse-python
""" > ext-env/Dockerfile

In [8]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_context = Environment(
    build=BuildContext(path="ext-env"),
    name="ext-env",
    description="Environment created from a Docker context.",
)
ml_client.environments.create_or_update(env_docker_context)

Run the training

create an AzureML job that executes the yolov8 training against the compute cluster we created earlier

In [2]:
from azure.ai.ml import command
from azure.ai.ml import Input

command_job = command(
    inputs=dict(
        data=Input(
            type="uri_folder",
            path="azureml:sigatoka-damage-ds:1",#Dataset path
        ),
         model_to_train=Input(
            type="custom_model",
            path="./yolov8x-seg.pt"
        )
    ),
    command="""
    echo "The data asset path is ${{ inputs.data }}" &&
    # Update custom.yaml to contain the correct path
    sed -i "s|path:.*$|path: ${{ inputs.data }}|" custom.yaml &&
    # Now custom.yaml contains the correct path so we can run the training
    yolo segment train data=custom.yaml model=${{ inputs.model_to_train }} epochs=300 imgsz=640 batch=16 project=Mask name=experiment
    """,
    code="./src/",
    environment="Yolov8-environment:1",
    compute="MangoDamage-dl-gpu", # change this line to match the name of your Compute
    experiment_name="sigatoka",
    display_name="sigatoka",
)

ml_client.create_or_update(command_job)


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Experiment,Name,Type,Status,Details Page
sigatoka,blue_kite_kdl776pn34,command,Starting,Link to Azure Machine Learning studio
